In [2]:
import numpy as np
import time
import mediapipe as mp
import cv2
import handtrackingmodule as htm
import math
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume

In [3]:
wcam,hcam=680,420
ctime=0
ptime=0
cap=cv2.VideoCapture(0)
cap.set(3,wcam)
cap.set(4,hcam)
detector=htm.handdetector()




devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(
    IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = interface.QueryInterface(IAudioEndpointVolume)
#volume.GetMute()
#volume.GetMasterVolumeLevel()
volumerange=volume.GetVolumeRange()
minvol=volumerange[0]
maxvol=volumerange[1]
vol=0
volbar=400
volper=0




while True:
    success,img=cap.read()
    img=detector.findHands(img)
    lmlist=detector.findposition(img,draw=False)
    if len(lmlist)!=0:
        #print(lmlist[4],lmlist[8])
        x1,y1=lmlist[4][1],lmlist[4][2]
        x2,y2=lmlist[8][1],lmlist[8][2]
        cx,cy=(x1+x2)//2, (y1+y2)//2
        cv2.circle(img,(x1,y1),15,[255,0,255],cv2.FILLED)
        cv2.circle(img,(x2,y2),15,[255,0,255],cv2.FILLED)
        cv2.line(img,(x1,y1), (x2,y2),(255,0,255), 2)
        cv2.circle(img, (cx,cy), 5, [255,255,0], cv2.FILLED)
        length=math.hypot(x2-x1,y2-y1)
        #print(length)
        # finger range 50-300
        # volume range -65-0
        vol=np.interp(length,[12,120],[minvol,maxvol])
        volbar=np.interp(length,[12,120],[400,150])
        volper=np.interp(length,[12,120],[0,100])
        volume.SetMasterVolumeLevel(vol, None)
        #print(vol)
        if length<50:
            cv2.circle(img,(cx,cy),15,(0,255,0),cv2.FILLED)
    cv2.rectangle(img,(50,150),(85,400),(250,0,0),3)
    cv2.rectangle(img,(50,int(volbar)),(85,400),(250,0,0),cv2.FILLED)
    cv2.putText(img,f'{int(volper)}%',(40,450),cv2.FONT_HERSHEY_SIMPLEX,1,(250,0,0),2)

    ctime=time.time()
    fps=1/ (ctime-ptime)
    ptime=ctime
    cv2.putText(img,f'FPS:{int(fps)}',(40,50),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2)
    cv2.imshow('img',img)
    cv2.waitKey(1)
